# Relatório de transmissão digital

Transmissão Digital
Prof: Diniz

Aluno: Fernando Dias

Esse relatório tem como objetivo implementar e expandir alguns dos resultados obtidos pelo artigo "Channel Estimation and Performance Analysis of One-Bit Massive MIMO Systems". 

A proposta é avaliar qual o desempenho de um sistema MIMO massivo onde cada antena utiliza um conversor analógico digital de apenas um bit. O processo de estimação de canal se dá por um novo processo proposto pelo artigo que consiste na linearização dos efeitos de quantização em uma expressão estatisticamente equivalente.

Esse documento é do tipo `notebook` e o código é interativo e pode ser executado na plataforma `Colab` do google. O link para acesso ao documento é: 

## 0 - Preparação

O primeiro passo é preparar as principais funções a serem utilizadas ao longo desse relatório. As funções consistem em:
* Gerar matrizes de canal e matrizes de bits para transmissão
* Realizar modulação e demodulação QPSK
* Realizar a função de quantização

Começaremos o código importando as bibliotecas necessárias:

In [1]:
# Bibliotecas de terceiros
import numpy as np # Realiza cálculos matriciais eficientes
import scipy as sp # Contém as funções científicas mais comuns
import pandas as pd # Cria tabelas para visualização de dados
import matplotlib.pyplot as plt # Forma os gráficos 
import seaborn as sns # Biblioteca para formação rápida de gráficos
import pulp as pl # Biblioteca para otimização linear

### Geração do canal e dos bits

A primeira função é a `GerarCanal(M,K)`. Ela recebe as dimensões do canal e retorna a matriz de canal correspondente. Ela é definida na forma:

$$GerarCanal(M,K)\to H\in\mathbb{C}^{M\times K}\sim\mathcal{CN}(0,1)$$

Já a segunda é a `GerarBits(K,tamanho)`. Ela gera uma matriz de bits com $K$ colunas e $t$ bits para transmissão. É definida na forma:

$$GerarBits(K,\tau)\to B\in[0,1]^{K\times\tau}$$

In [2]:
def GerarCanal(M,K):
    """
    GerarCanal(M,K)
    Descrição:
        Gera a matriz de canal M*K. Onde M é o número de antenas 
        receptoras e K o número de antenas transmissoras.
    Recebe:
        - M: Número de antenas receptoras
        - K: Número de antenas transmissoras
    Retorna:
        - Matriz complexa M*K 
    """
    return np.matrix(np.random.normal(0,1,size=(M,K))+1j*np.random.normal(0,1,size=(M,K)))

def GerarBits(K,tamanhoMensagem):
    """
    GerarCanal(M,K)
    Descrição:
        Gera a matriz de canal M*K. Onde M é o número de antenas 
        receptoras e K o número de antenas transmissoras.
    Recebe:
        - M: Número de antenas receptoras
        - K: Número de antenas transmissoras
    Retorna:
        - Matriz complexa M*K 
    """
    return np.matrix(np.random.choice([0,1],size=(K,tamanhoMensagem)))

### (De)modulação e quantização

A próxima etapa consiste na definição do modulador e do demodulador QPSK. É trivial assumir que, devido a quantização na recepção, essa é a modulação escolhida. Assim, o demodulador QPSK já tem o processo de decisão que consiste em avaliar o sinal das partes reais e imaginárias do sistema.

In [16]:

def ModuladorQPSK(s):
    """
    ModuladorQPSK(s)
    Descrição:
        Recebe uma matriz de bits e retorna uma matriz de símbolos modulada em QPSK
    Parâmetros:
        - s: Matriz de bits
    Retorna:
        - Matriz de símbolos QPSK de energia unitária
    """
    #if tuple(np.unique(s)) != (0,1):
    #    raise Exception("Deve receber matriz de bits (0 ou 1)")
        
    return (np.where(s==0,-1,1)[:,::2]+1j*np.where(s==0,-1,1)[:,1::2])/np.sqrt(2)

def DemoduladorQPSK(s):
    """
    DemoduladorQPSK(s)
    Descrição:
        Recebe uma matriz de símbolos complexos e retorna a matriz de bits correspondente
    Parâmetros:
        - s: Matriz de símbolos
    Retorna: 
        - Matriz de bits
    """
    return np.stack([
        np.where(np.real(s)>0,1,0), # Faz a decisão com base no sinal do número complexo
        np.where(np.imag(s)>0,1,0)
    ]).swapaxes(0,1).reshape(s.shape[0],-1,order='F')

Ao invés de explicar passo a passo como as funções acima funcionam, abaixo está uma prova de conceito que prova que os moduladores funcionam.

In [8]:
# Define as dimensões da matriz de bits
K = 1000
T = 10000

# Matriz de bits aleatória
s = np.random.choice([0,1],size=(K,T))

# Faz a modulação
s_mod = ModuladorQPSK(s)

# Faz a demodulação
s_demod = DemoduladorQPSK(s_mod)

# Retorna true apenas se todos os elementos de s_demod 
# sao iguais aos de s
np.equal(s_demod,s).all()

True

O próximo passo é a definição do quantizador. A única diferença entre o quantizador e o demodulador QPSK é que o quantizador mantém os símbolos complexos e não converte para bits. Aplicar o demodulador no resultado do quantizador tem o mesmo efeito de aplicar o demodulador diretamente.

In [9]:
def Quantizador(data):
    """
    Quantizador(signal)
    Descrição:
        Realiza a quantização do sinal recebido. 
        A quantização é equivalente à modulação QPSK.
    Parâmetros:
        - signal: O sinal (complexo) a ser quantizado. 
    Retorna:
        - Sinal quantizado. 
    """
    return (np.sign(np.real(data))+1j*np.sign(np.imag(data)))/np.sqrt(2)

### Cálculo dos receptores

Os receptores considerados no trabalho são dois: O _Maximum Ratio Combining_ (MRC) e o _Zero Forcing_ (ZF). Serão definidas duas funções que retornam os receptores com base na matriz de canal estimado.

In [10]:
def ReceptorZF(H_est,signal=None):
    """
    ReceptorZF(H_est,signal)
    Calcula o receptor ZF com base no `H_est` e o aplica no `signal`.
    Parâmetros:
        - H_est: Estimativa do canal para aplicar o receptor
        - signal (default=None): Sinal na qual processar o receptor
    Retorna:
        - Se tiver sinal, 
            - Retorna o sinal depois de aplicado o receptor
        - Se não tiver sinal,
            - Retorna a matriz do receptor
    """
    # Error detection
    if type(H_est) != np.matrix:
        raise Exception("H_est type must be matrix")
    
    # Calculo do receptor
    receptor = np.linalg.inv(np.matmul(H_est.H,H_est))
    receptor = np.matmul(receptor,H_est.H)
    
    if signal:
        # Aplicacao do receptor ao resultado
        return np.matmul(receptor[np.newaxis,:,:],np.swapaxes(signal,1,0)[:,:,np.newaxis])
    
    return receptor

def ReceptorMRC(H_est,signal=None):
    """
    ReceptorMRC(H_est,signal)
    Descrição:
        Calcula o receptor MRC com base no `H_est` e o aplica no `signal`, se dado. Caso
    Parâmetros:
        - H_est: Estimativa do canal para aplicar o receptor
        - signal (default=None): Sinal na qual processar o receptor
    Retorna:
        - Se tiver sinal, 
            - Retorna o sinal depois de aplicado o receptor
        - Se não tiver sinal,
            - Retorna a matriz do receptor
    """
    # Error detection
    if type(H_est) != np.matrix:
        raise Exception("H_est type must be matrix")
    
    # Calculo do receptor
    receptor = H_est.H
    
    if signal:
        # Aplicacao do receptor ao resultado
        return np.matmul(receptor[np.newaxis,:,:],np.swapaxes(signal,1,0)[:,:,np.newaxis])
    
    return receptor

### Transmissão MIMO

Finalmente, será definido o processo de transmissão MIMO. Para isso será definida uma função que simplesmente aplica a fórmula de transmissão:
$$\mathbf{y}=\sqrt{\rho}H\mathbf{s}+\mathbf{n}$$
Para cada execução dessa função, um novo valor de ruído AWGN é calculado.

In [12]:
def TransmissaoMIMO(H,s,SNR=15):
    """
    TransmissaoMIMO(H,s,power=1,N0=1)
    Descrição:
        Realiza uma transmissão MIMO pelo canal H especificado 
        e adiciona um ruído branco e gaussiano no resultado.
    Parâmetros:
        - H: Canal de dimensão M*K
        - s: Sinal com dimensões K*(message length)
        - SNR: Signal to Noise Ratio em dB (default 15)
    Retorna:
        - y: Sinal recebido de dimensões M*(message length)
    """
    # Cálculo do ruído a ser aplicado
    noise = np.random.normal(0,1,size=(H.shape[0],s.shape[1]))

    # Retorna resultado da transmissão
    return (np.sqrt(10**(SNR/20)))*np.matmul(H,s) + noise

Para demonstrar o que foi obtido até então. Será simulado um processo de comunicação MIMO para um caso simples, com $M=16$ antenas e $K=4$ usuários para diferentes níveis de SNR e será calculado o BER.

In [ ]:
# Definição dos parâmetros de simulação
M = 16
K = 4
tamanhoBits = 10**(7)
repeticoes = 10

resultados = pd.DataFrame([]) # Tabela para armazenamento dos resultados

# Repete a simulação para valores entre -20 e 20 de SNR, com passo 5
for SNR in range(-20,25,5):
    # Calcula o H_real
    H_real = GerarCanal(M,K)
    # Calcula os receptores ZF e MRC e coloca em um dicionário
    receptores = {"ZF":ReceptorZF(H_real),"MRC":ReceptorMRC(H_real)}
    # Repete mais 100 vezes a transmissão para alcançar valores maiores de BER
    BE = {"ZF":0,"MRC":0}
    for i in range(repeticoes):
        # Gera os bits para transmissão
        b = GerarBits(K,tamanhoBits)
        # Modula QPSK
        s = ModuladorQPSK(b)
        # Faz a transmissão pelo canal
        y = TransmissaoMIMO(H_real,s)
        # Faz a recepção do sinal recebido por ambos os receptores
        for receptor in ['ZF','MRC']:
            b_est = receptores[receptor]@y
            b_est = DemoduladorQPSK(b_est)
            BE[receptor] += np.sum(np.abs(b_est-b))
    # Calcula o BER     
    BE['ZF'] /=(repeticoes*K*tamanhoBits)
    BE['MRC'] /=(repeticoes*K*tamanhoBits)
    resultados = pd.concat([resultados,pd.DataFrame(
        {
            'SNR':[SNR,SNR],
            'Receptor':["ZF","MRC"],
            "BER":[BE['ZF'],BE['MRC']]
        }
    )],ignore_index=True)

In [ ]:
sns.lineplot(data=resultados,x="SNR",y="BER",hue="Receptor")
plt.yscale('log')

In [ ]:
resultados

## 1 - Estimação de 